In [1]:
from estnltk import Text
a = Text('Kui Arno isaga koolimajja jõudis, olid tunnid juba alanud.')
a.text

'Kui Arno isaga koolimajja jõudis, olid tunnid juba alanud.'

In [2]:
a.tag_layer(["morph_analysis"])

Text(text="Kui Arno isaga koolimajja jõudis, olid tunnid juba alanud.")

In [3]:
def add_spans (text, span_name):
    span = getattr(text, span_name)
    spancount = len(span.spans)
    spanstarts = []
    spanends = []
    text=text.text
    
    for i in range(spancount):
        spanstart = span[i].start
        spanend = span[i].end
        spanstarts.append(spanstart)
        spanends.append(spanend)
    
    for i in range(spancount):
        text=''.join([text[:spanstarts[i]+i*41],"<span onclick=box(this.innerHTML)>",
            text[spanstarts[i]+i*41:spanends[i]+i*41],"</span>",text[spanends[i]+i*41:len(text)]])
    return text

In [4]:
t = '''
<html>
<style>
.boxes {{
    background: linear-gradient(to bottom right, white, lightgray);
    border: 1px solid black;
    border-radius: 4px;
    box-shadow: 4px 4px 4px gray;
    width: fit-content;
}}
</style>
<body>

<p id="maintext">{}</p>

<script>

function box(content) {{
    let textbox = document.createElement("DIV");
    textbox.classList.add("boxes");
    let t = document.createTextNode(content);
    textbox.appendChild(t);
    let textNode = document.getElementById("maintext");
    let previousBox = document.querySelector('.boxes');
    if (previousBox){{
        previousBox.parentNode.removeChild(previousBox);
    }}
    textNode.appendChild(textbox);
}}

</script>

</body>
</html>
'''


In [5]:
from IPython.display import display_html

def display_layer(text,layer):
    result=add_spans(text,layer)
    display_html(t.format(result), raw=True)
    
display_layer(a,"morph_analysis")

Kui Arno isaga koolimajja jõudis , olid tunnid juba alanud .